# Connecting to the board


- Imports
- Load data sample

In [1]:
import samna
print(f'Samna version {samna.__version__}')

import numpy as np
import matplotlib.pyplot as plt
from rockpool.devices.xylo.syns65302 import config_from_specification as config_from, mapper as mapper, XyloSamna as XyloSamna, XyloMonitor as XyloMonitor
from rockpool.devices.xylo.syns65302 import xa3_devkit_utils as hdu

from rockpool.transform.quantize_methods import channel_quantize

Samna version 0.39.7.1+ge088b084


In [2]:
from rockpool.nn.networks import SynNet
from rockpool.nn.modules import LIFTorch
import warnings
warnings.filterwarnings("ignore")

ckpt = '../../../docs/devices/xylo-a3/model_sample/to_deploy_inXylo.json'

# - Loading trained model architecture parameters
arch_params = {'n_classes': 1,
'n_channels': 16,
'size_hidden_layers':[63, 63, 63],
'time_constants_per_layer':[3,7,7],
'tau_syn_base': 0.02,
'tau_mem': 0.02,
'tau_syn_out': 0.02,
'neuron_model': LIFTorch,
'dt': 0.00994,
'output': 'vmem'}

# - Instantiating the model backbone and loading trained checkpoint
model = SynNet(** arch_params)
model.load(ckpt)

# - Getting the connected devices and choosing xyloa3 board
xylo_node = hdu.find_xylo_a3_boards()[0]


WARNING    /home/vleite/SynSense/rockpool/rockpool/nn/networks/__init__.py:15: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
 [py.warnings]
  warnings.warn(f"{err}")

WARNING    /home/vleite/SynSense/rockpool/rockpool/nn/networks/__init__.py:20: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
 [py.warnings]
  warnings.warn(f"{err}")



## Clock frequencies

In [3]:
clock_frequencies = [50, 25, 12.5, 6.25]

# Power measurement - iddle - no network

In [4]:
dt = 0.01 # seconds
duration = 3 # seconds

power_consumption_io = []
power_consumption_analog = []
power_consumption_digital = []

# - For every clock rate we want to measure
for clock in clock_frequencies:

    io_power = []
    analog_power = []
    digital_power = []

    # Instantiate XyloMonitor and call evolve for three second in silence
    xylo_monitor = XyloMonitor(device=xylo_node, main_clk_rate=clock, config=None, power_frequency=100, dt=dt, output_mode='Spike')

    # - Run ten times to average all samples
    for i in range(20):
        out, state, rec = xylo_monitor.evolve(read_timeout=duration, record_power=True)

        io_power.append(np.mean(rec['io_power']))
        analog_power.append (np.mean(rec['analog_power']))
        digital_power.append(np.mean(rec['digital_power']))

    xylo_monitor = []
    power_consumption_io+=[np.mean(io_power)]
    power_consumption_analog+=[np.mean(analog_power)]
    power_consumption_digital+=[np.mean(digital_power)]


for clock, io, analog, digital in zip(clock_frequencies, power_consumption_io, power_consumption_analog, power_consumption_digital):
    print(f'Clock:\t\t{clock} MHz\n io:\t\t{io*1000:.1f} mW \t\t AFE core:\t{analog*1000:.1f} mW\t\t SNN core logic:\t{digital*1000:.1f} mW')

Clock:		50 MHz
 io:		2.0 mW 		 AFE core:	0.0 mW		 SNN core logic:	2.8 mW
Clock:		25 MHz
 io:		1.9 mW 		 AFE core:	0.0 mW		 SNN core logic:	1.5 mW
Clock:		12.5 MHz
 io:		1.9 mW 		 AFE core:	0.0 mW		 SNN core logic:	0.9 mW
Clock:		6.25 MHz
 io:		1.8 mW 		 AFE core:	0.0 mW		 SNN core logic:	0.6 mW


# Power measurement - iddle - with loaded network

In [5]:
clock_frequencies = [50, 25, 12.5]

In [6]:
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper
import rockpool.transform.quantize_methods as q

# getting the model specifications using the mapper function
spec = mapper(model.as_graph(), weight_dtype='float', threshold_dtype='float', dash_dtype='float')
# quantizing the model
spec.update(q.channel_quantize(**spec))

xylo_conf, is_valid, msg = config_from_specification(**spec)
xylo_conf.operation_mode = samna.xyloAudio3.OperationMode.RealTime

dt = 0.01 # seconds
duration = 3 # seconds

power_consumption_io = []
power_consumption_analog = []
power_consumption_digital = []

# - For every clock rate we want to measure
for clock in clock_frequencies:

    io_power = []
    analog_power = []
    digital_power = []

    # Instantiate XyloMonitor and call evolve for three second in silence
    xylo_monitor = XyloMonitor(device=xylo_node, main_clk_rate=clock, config=xylo_conf, power_frequency=100, dt=dt, output_mode='Spike')

    # - Run ten times to average all samples
    for i in range(20):
        out, state, rec = xylo_monitor.evolve(read_timeout=duration, record_power=True)

        io_power.append(np.mean(rec['io_power']))
        analog_power.append (np.mean(rec['analog_power']))
        digital_power.append(np.mean(rec['digital_power']))

    xylo_monitor = []

    power_consumption_io+=[np.mean(io_power)]
    power_consumption_analog+=[np.mean(analog_power)]
    power_consumption_digital+=[np.mean(digital_power)]


for clock, io, analog, digital in zip(clock_frequencies, power_consumption_io, power_consumption_analog, power_consumption_digital):
    # print(power)
    print(f'Clock:\t\t{clock} MHz\n io:\t\t{io*1000:.1f} mW \t\t AFE core:\t{analog*1000:.1f} mW\t\t SNN core logic:\t{digital*1000:.1f} mW')

Clock:		50 MHz
 io:		2.0 mW 		 AFE core:	0.0 mW		 SNN core logic:	3.5 mW
Clock:		25 MHz
 io:		1.9 mW 		 AFE core:	0.0 mW		 SNN core logic:	1.9 mW
Clock:		12.5 MHz
 io:		1.7 mW 		 AFE core:	0.0 mW		 SNN core logic:	1.1 mW


# Power measurement - inference over a trained network

In [7]:
from scipy.io import wavfile
!pip install simpleaudio
import simpleaudio as sa
import numpy as np

def get_wave_object(test_file):
    sample_rate, data = wavfile.read(test_file)

    duration = int(len(data)/sample_rate) # in seconds
    n = data.ndim

    if data.dtype == np.int8:
        bytes_per_sample = 1
    elif data.dtype == np.int16:
        bytes_per_sample = 2
    elif data.dtype == np.float32:
        bytes_per_sample = 4
    else:
        raise ValueError("recorded audio should have 1 or 2 bytes per sample!")

    wave_obj = sa.WaveObject(
        audio_data= data,
        num_channels=data.ndim,
        bytes_per_sample=bytes_per_sample,
        sample_rate=sample_rate
    )

    return duration,wave_obj

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper
import rockpool.transform.quantize_methods as q

# getting the model specifications using the mapper function
spec = mapper(model.as_graph(), weight_dtype='float', threshold_dtype='float', dash_dtype='float')
# quantizing the model
spec.update(q.channel_quantize(**spec))

xylo_conf, is_valid, msg = config_from_specification(**spec)
xylo_conf.operation_mode = samna.xyloAudio3.OperationMode.RealTime

dt = 0.01 # seconds
duration = 3 # seconds

power_consumption_io = []
power_consumption_analog = []
power_consumption_digital = []

# - For every clock rate we want to measure
for clock in clock_frequencies:

    io_power = []
    analog_power = []
    digital_power = []

    # Instantiate XyloMonitor and call evolve for three seconds playing an audio
    xylo_monitor = XyloMonitor(device=xylo_node, main_clk_rate=clock, config=xylo_conf, power_frequency=100, dt=dt, output_mode='Spike')

    # - Run ten times to average all samples
    for i in range(20):
        test_audio = '../../../docs/devices/xylo-a3/audio_sample/cry_sample_3sec.wav'
        duration, wave_obj = get_wave_object(test_audio)
        
        play_obj = wave_obj.play()
        out, state, rec = xylo_monitor.evolve(read_timeout=duration, record_power=True)
        play_obj.wait_done()
        
        print(f'cry detected: {np.sum(out)>0}')

        io_power.append(np.mean(rec['io_power']))
        analog_power.append (np.mean(rec['analog_power']))
        digital_power.append(np.mean(rec['digital_power']))


    xylo_monitor = []

    power_consumption_io+=[np.mean(io_power)]
    power_consumption_analog+=[np.mean(analog_power)]
    power_consumption_digital+=[np.mean(digital_power)]


for clock, io, analog, digital in zip(clock_frequencies, power_consumption_io, power_consumption_analog, power_consumption_digital):
    # print(power)
    print(f'Clock:\t\t{clock} MHz\n io:\t\t{io*1000:.1f} mW \t\t AFE core:\t{analog*1000:.1f} mW\t\t SNN core logic:\t{digital*1000:.1f} mW')

cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
cry detected: False
